<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para UdeA

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [2]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j

Overwriting drive.cfg


##  Load data bases

In [4]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [5]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [6]:
#from check_quality import *
#check_quality(UDEA)

## Indices:
Información obtenida de la columna: `json_column='UDEA_authors'`

In [7]:
json_column='UDEA_authors'

Que contiene listas de diccionarios con la información del autor UDEA: 

`{'DEPARTAMENTO': 'Instituto de Biología',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Sin Grupo Asociado',
  'INICIALES': 'I.',
  'NOMBRE COMPLETO': 'Idalyd Fonseca Gonzalez',
  'NOMBRES': 'Idalyd',
  'PRIMER APELLIDO': 'Fonseca',
  'SEGUNDO APELLIDO': 'Gonzalez',
  'WOS_affiliation': ['Univ Antioquia, Colombia.'],
  'WOS_author': ['FONSECA, IDALYD',
   'FONSECA-GONZALEZ, IDALYD',
   'Fonseca-Gonzalez, Idalyd',
   'Fonseca-Gonzalez, I.'],
  'full_name': 'FONSECA GONZALEZ IDALYD'}`

Otras columnas: `['OA','Z9'*,SCP_Cited by']`, `*`: WOS cited by

Ver también [WOS field tags](https://images.webofknowledge.com/images/help/WOS/hs_wos_fieldtags.html)

# Resultados totales

Artículos no identificados:

In [8]:
UDEA_NOT=UDEA[UDEA[json_column]==''].reset_index(drop=True)
UDEA_NOT.shape[0]

4019

Artículos identificados

In [9]:
UDEA_YES=UDEA[UDEA[json_column]!=''].reset_index(drop=True)
UDEA_YES.shape[0]

11681

### Análisis sobre artículos identificados

In [10]:
def flatten_if_nested(l):
    flatten=False
    for i in l:
        if type(i)==list:
            #return i
            flatten=True
    if flatten:
        l=[item for sublist in l for item in sublist]
        l=pd.np.array(l)
    return l
def extract_key(df,key,json_column='UDEA_authors'):
    '''
    Extract all the unique key values of the list of dictionaries in 
    a json column when the key value is a string or another list
    '''
    ll=df[json_column].apply(lambda l: np.unique([ d.get(key) for d in l 
                                if d.get(key) ]) if type(l)==list else l)
    if ll.str[0].apply(lambda l: l if type(l)==list else None).dropna().shape[0]:
        ll=ll.apply(flatten_if_nested)
    ll=ll.apply(pd.Series).stack().values
    return pd.DataFrame( {key:list(ll)} ).groupby(key)[key].count().sort_values(ascending=False)

In [11]:
extract_key(UDEA_YES,'FACULTAD')

FACULTAD
Facultad de Medicina                        3077
Facultad de Ciencias Exactas y Naturales    2255
Facultad de Ingeniería                      1778
Facultad de Ciencias Agrarias                656
Facultad de Ciencias Sociales y Humanas      215
Facultad de Artes                             13
Name: FACULTAD, dtype: int64

In [12]:
extract_key(UDEA_YES,'DEPARTAMENTO')

DEPARTAMENTO
Departamento de Microbiología y Parasitología                   886
Instituto de Física                                             862
Instituto de Investigaciones Médicas                            717
Instituto de Biología                                           656
Instituto de Química                                            644
Departamento de Medicina Interna                                642
Departamento de  Producción Agropecuaria                        406
Departamento de Pediatría y Puericultura                        388
Departamento de Ingeniería Metalúrgica                          334
Departamento de Ingeniería Sanitaria  y Ambiental               317
Escuela de Medicina Veterinaria                                 305
Departamento de Ingeniería Mecánica                             279
Departamento de Ingeniería Quimica                              276
Departamento de Cirugía                                         210
Departamento de Fisiología         

In [13]:
extract_key(UDEA_YES,'GRUPO')

GRUPO
Sin Grupo Asociado                                                                                                                                                                                436
Grupo de Materia Condensada-UdeA                                                                                                                                                                  261
Inmunovirología                                                                                                                                                                                   236
Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección                                                                         232
Grupo de Manejo Eficiente de la Energía, GIMEL                                                                                                                                                    219
Grup

In [14]:
extract_key(UDEA_YES,'full_name')

full_name
DUQUE ECHEVERRI CARLOS ALBERTO        261
BEDOYA BERRIO GABRIEL DE JESUS        123
LOPERA RESTREPO FRANCISCO JAVIER      120
CERON MUÑOZ MARIO FERNANDO            120
RUGELES LOPEZ MARIA TERESA            116
JAIMES BARRAGAN FABIAN ALBERTO        113
CARMONA FONSECA JAIME DE JESUS        113
PEÑUELA MESA GUSTAVO ANTONIO          110
VELEZ BERNAL IVAN DARIO               109
OLIVERA ANGEL MARTHA EUFEMIA          101
AMARILES MUÑOZ PEDRO JOSE              95
RIOS LUIS ALBERTO                      92
RESTREPO BETANCUR LUIS FERNANDO        92
CARDONA MAYA WALTER DARIO              91
CARDONA ARIAS JAIBERTH ANTONIO         91
ARDILA MEDINA CARLOS MARTIN            86
MORALES ARAMBURO ALVARO LUIS           84
BLAIR TRUJILLO SILVIA VICTORIA         82
ROBLEDO RESTREPO SARA MARIA            80
MONDRAGON PEREZ FANOR                  79
CADAVID JARAMILLO ANGELA PATRICIA      78
AGUDELO SUAREZ ANDRES ALONSO           77
CORNEJO OCHOA JOSE WILLIAM             75
TRIANA CHAVEZ OMAR      

# Búsquedas

In [15]:
def extract_key_unique(*args,**kwargs):
    keys=extract_key(*args,**kwargs).keys()
    return [ k for k in keys if k]

def get_groups(l,g):
    for d in l:
        gt=d.get('GRUPO')
        if gt and type( gt )==str:
            gs=gt.replace(
                ', Grupo','; Grupo'
            ).split('; ')
            for gg in gs:
                if gg not in g:
                    g.append(gg)
    return g

facultades={'key':'FACULTAD',
            'values' : extract_key_unique(UDEA,'FACULTAD',json_column='UDEA_authors') }
departamentos={'key':'DEPARTAMENTO',
            'values' :extract_key_unique(UDEA,'DEPARTAMENTO',json_column='UDEA_authors')}
nombre_completo={'key'    : 'NOMBRE COMPLETO',
            'values' : extract_key_unique(UDEA,'NOMBRE COMPLETO',json_column='UDEA_authors')}
full_name={'key'    : 'full_name',
            'values' : extract_key_unique(UDEA,'full_name',json_column='UDEA_authors')}
udea_affiliations={'key'    : 'WOS_affiliation',
            'values' : extract_key_unique(UDEA,'WOS_affiliation',json_column='UDEA_authors')}
wos_affiliations={'key'    : 'affiliation',
            'values' : extract_key_unique(UDEA,'WOS_affiliation',json_column='authors_WOS')}
udea_author={'key'    : 'WOS_author',
            'values' : extract_key_unique(UDEA,'WOS_author',json_column='UDEA_authors')}
wos_author={'key'    : 'WOS_author',
            'values' : extract_key_unique(UDEA,'WOS_author',json_column='authors_WOS')}


#.apply(....) is a loop!
g=[]
#append to g
tmp=UDEA.UDEA_authors.apply(lambda l: 
                        get_groups(l,g)
        if type(l)==list else None
                        )
grupos={'key':'GRUPO',
            'values' :g}


## Función de búsqueda

For value string or list of each dictionary within a list of dictionaries, like the column 'UDEA_authors' in `UDEA` DataFrame

In [80]:
def query_json_column(q,df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,score_cutoff=0):
    #Found best exact match from index
    fchoices=fwp.extractOne(q,choices['values'],scorer=scorer,score_cutoff=score_cutoff)
    # Exact search in indexed subcolumn converted to strins (e.g list → string if necessary)
    if fchoices:
        fchoices=fchoices[0]
        dfF=df[df[json_column].apply(lambda l: True in [ str(d.get(choices['key'])).find(fchoices)>-1 
                                        for d in l if d.get(choices['key'])] if type(l)==list else False)]
        return dfF.reset_index(drop=True)
    else:
        return pd.DataFrame()

### Autor

In [17]:
r=query_json_column('Diego Alejandro Restrepo Quintero',df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [18]:
r.shape

(37, 181)

In [417]:
#r[['TI','AU','authors_WOS',json_column]].reset_index(drop=True)[5:7]

## Grupos

Ejemplo

In [20]:
r=query_json_column('Grupo de Fenomenología de Interacciones Fundamentales',df=UDEA,json_column='UDEA_authors',
                        choices=grupos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [21]:
r.shape

(82, 181)

Buscar todos

In [22]:
gdf=pd.DataFrame()
for g in grupos['values']:
    r=query_json_column(g,df=UDEA,json_column='UDEA_authors',choices=grupos,
                        scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)
                        
    gdf=gdf.append( {'Group':g,'articles':r.shape[0]},ignore_index=True )
gdf['articles']=gdf['articles'].astype(int)

In [23]:
gdf.sort_values('articles',ascending=False).reset_index(drop=True)[:10]

,Group,articles
0,Sin Grupo Asociado,436
1,Grupo de Materia Condensada-UdeA,300
2,"Grupo Reproducción, Inmunovirología, Infección y Cáncer",284
3,Inmunovirología,284
4,Grupo de Estado Sólido,272
5,"Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección,",252
6,"Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección",252
7,Grupo Académico de Epidemiología Clínica,240
8,"Grupo Académico de Epidemiología Clínica, Nacer, Salud Sexual y Reproductiva",240
9,Grupo de Neurociencias de Antioquia,240


## Departamento

In [24]:
r=query_json_column('Instituto de Física',df=UDEA,json_column='UDEA_authors',
                        choices=departamentos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [25]:
r.shape

(862, 181)

## Centro

Ejemplo

In [26]:
cen=query_json_column('Facultad de Ciencias Exactas y Naturales',df=UDEA,json_column='UDEA_authors',
                        choices=facultades,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [27]:
cen.shape

(2255, 181)

Todos

In [28]:
fdf=pd.DataFrame()
for f in facultades['values']:
    r=query_json_column(f,df=UDEA,json_column='UDEA_authors',choices=facultades,
                        scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)
    fdf=fdf.append( {'Facultad':f,'articles':r.shape[0]},ignore_index=True )
fdf['articles']=fdf['articles'].astype(int)

In [29]:
fdf.sort_values('articles',ascending=False)

,Facultad,articles
0,Facultad de Medicina,3077
1,Facultad de Ciencias Exactas y Naturales,2255
2,Facultad de Ingeniería,1778
3,Facultad de Ciencias Agrarias,656
4,Facultad de Ciencias Sociales y Humanas,215
5,Facultad de Artes,13


## Citas

In [30]:
UDEA_YES.sort_values('Z9',ascending=False)[['Z9','TI','SO','AU','PY']].reset_index(drop=True)[:10]

,Z9,TI,SO,AU,PY
0,3610,"An integrated map of genetic variation from 1,092 human genomes",NATURE,"Altshuler, DM\nDurbin, RM\nAbecasis, GR\nBentley, DR\nChakravarti, A\nClark, AG\nDonnelly, P\nEichler, EE\nFlicek, P\nGabriel, SB\nGibbs, RA\nGreen, ED\nHurles, ME\nKnoppers, BM\nKorbel, JO\nLande...",2012
1,1526,Leishmaniasis Worldwide and Global Estimates of Its Incidence,PLOS ONE,"Alvar, J\nVelez, ID\nBern, C\nHerrero, M\nDesjeux, P\nCano, J\nJannin, J\nden Boer, M\n",2012
2,1271,A global reference for human genetic variation,NATURE,"Altshuler, DM\nDurbin, RM\nAbecasis, GR\nBentley, DR\nChakravarti, A\nClark, AG\nDonnelly, P\nEichler, EE\nFlicek, P\nGabriel, SB\nGibbs, RA\nGreen, ED\nHurles, ME\nKnoppers, BM\nKorbel, JO\nLande...",2015
3,901,Human papillomavirus genotype attribution in invasive cervical cancer: a retrospective cross-sectional worldwide study,LANCET ONCOLOGY,"de Sanjose, S\nQuint, WGV\nAlemany, L\nGeraets, DT\nKlaustermeier, JE\nLloveras, B\nTous, S\nFelix, A\nBravo, LE\nShin, HR\nVallejos, CS\nde Ruiz, PA\nLima, MA\nGuimera, N\nClavero, O\nAlejo, M\nL...",2010
4,711,The Genome Sequence of Taurine Cattle: A Window to Ruminant Biology and Evolution,SCIENCE,"Elsik, CG\nTellam, RL\nWorley, KC\nGibbs, RA\nAbatepaulo, ARR\nAbbey, CA\nAdelson, DL\nAerts, J\nAhola, V\nAlexander, L\nAlioto, T\nAlmeida, IG\nAmadio, AF\nAnatriello, E\nAntonarakis, SE\nAnzola,...",2009
5,601,GENETIC ABSOLUTE DATING BASED ON MICROSATELLITES AND THE ORIGIN OF MODERN HUMANS,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF\nAMERICA,"GOLDSTEIN, DB\nLINARES, AR\nCAVALLISFORZA, LL\nFELDMAN, MW\n",1995
6,474,Mutations in IRF6 cause Van der Woude and popliteal pterygium syndromes,NATURE GENETICS,"Kondo, S\nSchutte, BC\nRichardson, RJ\nBjork, BC\nKnight, AS\nWatanabe, Y\nHoward, E\nde Lima, RLLF\nDaack-Hirsch, S\nSander, A\nMcDonald-McGinn, DM\nZackai, EH\nLammer, EJ\nAylsworth, AS\nArdinge...",2002
7,429,Leptogenesis,PHYSICS REPORTS-REVIEW SECTION OF PHYSICS LETTERS,"Davidson, S\nNardi, E\nNir, Y\n",2008
8,410,Temperature sensitivity of drought-induced tree mortality portends increased regional die-off under global-change-type drought,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF\nAMERICA,"Adams, HD\nGuardiola-Claramonte, M\nBarron-Gafford, GA\nVillegas, JC\nBreshears, DD\nZou, CB\nTroch, PA\nHuxman, TE\n",2009
9,376,Electron localization following attosecond molecular photoionization,NATURE,"Sansone, G\nKelkensberg, F\nPerez-Torres, JF\nMorales, F\nKling, MF\nSiu, W\nGhafur, O\nJohnsson, P\nSwoboda, M\nBenedetti, E\nFerrari, F\nLepine, F\nSanz-Vicario, JL\nZherebtsov, S\nZnakovskaya, ...",2010


In [31]:
UDEA_YES.Z9.sum()

75281

In [32]:
UDEA_YES.sort_values('SCP_Cited by',ascending=False)[[
    'SCP_Cited by','TI','SO','AU','PY']].reset_index(drop=True)[:10]

,SCP_Cited by,TI,SO,AU,PY
0,1586,Leishmaniasis Worldwide and Global Estimates of Its Incidence,PLOS ONE,"Alvar, J\nVelez, ID\nBern, C\nHerrero, M\nDesjeux, P\nCano, J\nJannin, J\nden Boer, M\n",2012
1,1160,"Effects of tranexamic acid on death, vascular occlusive events, and blood transfusion in trauma patients with significant haemorrhage (CRASH-2): A randomised, placebo-controlled trial",The Lancet,"Olldashi F., Kerçi M., Zhurda T., Ruçi K., Banushi A., Traverso M.S., Jiménez J., Balbi J., Dellera C., Svampa S., Quintana G., Piñero G., Teves J., Seppelt I., Mountain D., Hunter J., Balogh Z., ...",2010
2,994,Human papillomavirus genotype attribution in invasive cervical cancer: a retrospective cross-sectional worldwide study,LANCET ONCOLOGY,"de Sanjose, S\nQuint, WGV\nAlemany, L\nGeraets, DT\nKlaustermeier, JE\nLloveras, B\nTous, S\nFelix, A\nBravo, LE\nShin, HR\nVallejos, CS\nde Ruiz, PA\nLima, MA\nGuimera, N\nClavero, O\nAlejo, M\nL...",2010
3,626,The Genome Sequence of Taurine Cattle: A Window to Ruminant Biology and Evolution,SCIENCE,"Elsik, CG\nTellam, RL\nWorley, KC\nGibbs, RA\nAbatepaulo, ARR\nAbbey, CA\nAdelson, DL\nAerts, J\nAhola, V\nAlexander, L\nAlioto, T\nAlmeida, IG\nAmadio, AF\nAnatriello, E\nAntonarakis, SE\nAnzola,...",2009
4,598,GENETIC ABSOLUTE DATING BASED ON MICROSATELLITES AND THE ORIGIN OF MODERN HUMANS,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF\nAMERICA,"GOLDSTEIN, DB\nLINARES, AR\nCAVALLISFORZA, LL\nFELDMAN, MW\n",1995
5,485,Mutations in IRF6 cause Van der Woude and popliteal pterygium syndromes,NATURE GENETICS,"Kondo, S\nSchutte, BC\nRichardson, RJ\nBjork, BC\nKnight, AS\nWatanabe, Y\nHoward, E\nde Lima, RLLF\nDaack-Hirsch, S\nSander, A\nMcDonald-McGinn, DM\nZackai, EH\nLammer, EJ\nAylsworth, AS\nArdinge...",2002
6,439,The importance of early treatment with tranexamic acid in bleeding trauma patients: An exploratory analysis of the CRASH-2 randomised controlled trial,The Lancet,"Olldashi F., Kerçi M., Zhurda T., Ruçi K., Banushi A., Traverso M.S., Jiménez J., Balbi J., Dellera C., Svampa S., Quintana G., Piñero G., Teves J., Seppelt I., Mountain D., Balogh Z., Zaman M., D...",2011
7,432,Leptogenesis,PHYSICS REPORTS-REVIEW SECTION OF PHYSICS LETTERS,"Davidson, S\nNardi, E\nNir, Y\n",2008
8,424,Temperature sensitivity of drought-induced tree mortality portends increased regional die-off under global-change-type drought,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF\nAMERICA,"Adams, HD\nGuardiola-Claramonte, M\nBarron-Gafford, GA\nVillegas, JC\nBreshears, DD\nZou, CB\nTroch, PA\nHuxman, TE\n",2009
9,405,THE STRUCTURE OF THE PRESENILIN-1 (S182) GENE AND IDENTIFICATION OF 6 NOVEL MUTATIONS IN EARLY-ONSET AD FAMILIES,NATURE GENETICS,"CLARK, RF\nHUTTON, M\nFULDNER, RA\nFROELICH, S\nKARRAN, E\nTALBOT, C\nCROOK, R\nLENDON, C\nPRIHAR, G\nHE, C\nKORENBLAT, K\nMARTINEZ, A\nWRAGG, M\nBUSFIELD, F\nBEHRENS, MI\nMYERS, A\nNORTON, J\nMOR...",1995


In [33]:
UDEA_YES['SCP_Cited by'].sum()

78299

# TMP

In [273]:
aun=extract_key(UDEA_NOT,'WOS_author',json_column='authors_WOS')
aun[:3]

WOS_author
Builes, J. J.    27
                 25
Restrepo, A.     19
Name: WOS_author, dtype: int64

In [275]:
aun=aun.keys()

In [58]:
posib=extract_key(UDEA_YES,'WOS_author',json_column='authors_WOS').keys()

### Goods: i=2,3,4,6
### Bad: 1,5

In [1374]:
i=4
n=aun[i]
n

'Montegranario, Hebert'

In [1375]:
# if nold:
qq=query_json_column(n,df=UDEA_NOT,json_column='authors_WOS',
                        choices=wos_author,scorer=fuzz.ratio,score_cutoff=100)

In [1376]:
qq.index

RangeIndex(start=0, stop=9, step=1)

In [1377]:
for i in qq.index:
    print( [ d for d in qq.loc[i,'authors_WOS'] if n in d.get('WOS_author')] )

[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Montegranario, Hebert'}]
[{'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia

In [1378]:
qq.SO.unique()

array(['VARIATIONAL REGULARIZATION OF 3D DATA: EXPERIMENTS WITH MATLAB'],
      dtype=object)

In [1379]:
qq.shape

(9, 182)

In [1380]:
extract_key(qq[:1],'WOS_author',json_column='authors_WOS')

WOS_author
Montegranario, Hebert    1
Name: WOS_author, dtype: int64

In [1381]:
qq.loc[0,'authors_WOS']

[{'WOS_author': 'Montegranario, Hebert',
  'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'],
  'i': 0}]

In [1382]:
qq.loc[0,'SO']

'VARIATIONAL REGULARIZATION OF 3D DATA: EXPERIMENTS WITH MATLAB'

## Include SO

In [1383]:
aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json').reset_index(drop=True)

In [1384]:
def build_institutional_authors(x,author_df,x_author_key='WOS_author',x_affiliation_key='affiliation',
                                        author_key='WOS_author',
                                        affiliation_key='WOS_affiliation'):
    if type(x)!=list:
        return None
    ll=[]
    for j in range(len(x)):
        
                                #author_WOS→affiliation always have single affiliation
        kk=find_author_affiliation(x[j].get(x_author_key),x[j].get(x_affiliation_key)[0],
                                        author_df=author_df,
                                        author_key=author_key,
                                        affiliation_key=affiliation_key,
                                        ratio=0.9 )
        if kk:
            ll.append(kk)
    if not ll:
        ll=None
    return ll

In [1385]:
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
#UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)
df2=aunly.copy()
df2=pd.DataFrame( list( df2['UDEA_authors'].values ) )
df2['UDEA_authors']=aunly['UDEA_authors']
contents=df2[['WOS_author','WOS_affiliation','UDEA_authors']].reset_index(drop=True)
contents['WOS_author']=contents['WOS_author']#.astype(str)
contents['WOS_affiliation']=contents['WOS_affiliation']#.astype(str)

# ==============

In [1386]:
dfnot=qq#UDEA_NOT.copy()
dfnot=dfnot.reset_index(drop=True)

In [1387]:
l=dfnot['authors_WOS'].loc[0]

In [1388]:
l

[{'WOS_author': 'Montegranario, Hebert',
  'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'],
  'i': 0}]

In [1389]:
#for i in range(20):
#l=dfnot['authors_WOS'].loc[i]
#def json_fuzzy_merge(l,contents,right_target='UDEA_authors',
                       #left_on='WOS_author',extra_left_on='affiliation',
                       #right_on='WOS_author',extra_right_on='WOS_affiliation',
                       #cutoff=95,cutoff_extra=65,scorer=fuzz.partial_ratio):
if True:                
    right_target='UDEA_authors'
    left_on='WOS_author'
    extra_left_on='affiliation'
    right_on='WOS_author' 
    extra_right_on='WOS_affiliation'
    cutoff=92
    cutoff_extra=70
    scorer=fuzz.token_set_ratio                
    newl=[]
    for d in l:
        au=d.get(left_on)
        aff=d.get(extra_left_on)[0]
        break
        # extract best WOS author match
        #r=fwp.extractOne(au,contents[right_on],scorer=scorer)
        #if r[1]>=cutoff:
        #    raf=fwp.extractOne( aff, contents.loc[r[2],extra_right_on],scorer=scorer )
            #print(r[1],r[2],raf[1],aff,',',contents.loc[r[2],extra_right_on])
            #if raf[1]>=cutoff_extra:
            #    newl=newl+[  contents.loc[r[2],right_target]  ]
            #else:
                #check SO
                #newl=newl+[  contents.loc[r[2],right_target]  ]
        #break
    #if newl:
    #    return newl
    #else:
    #    return None

In [1390]:
au

'Montegranario, Hebert'

In [1391]:
aff

'Univ Antioquia, Math Inst, Medellin, Colombia.'

In [1392]:
Q=1
rau=fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=scorer)
rau

('Montegranario, H.', 93)

In [1393]:
#if rau[1]<cutoff:
RESET=False
tmprau=fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),
                   scorer=fuzz.partial_token_sort_ratio)
print(tmprau)
if RESET:
    Q=Q-0.1
    rau=tmprau

('Montegranario, H.', 93)


In [1394]:
#fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=fuzz.token_sort_ratio)
#fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=fuzz.ratio)#
#fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=fuzz.partial_token_set_ratio)

In [1395]:
if rau[1]>=cutoff:
    raf=contents[contents[right_on].apply( lambda l: rau[0] in l )].reset_index(drop=True)#.loc[0]

In [1396]:
raf[extra_right_on].loc[0]

['Univ Antioquia UdeA, Inst Matemat, Fac Ciencias Exactas & Nat, Medellin, Colombia.',
 'Univ Antioquia UdeA, Inst Fis, Fac Ciencias Exactas & Nat, Grp Mat Condensada UdeA, Medellin, Colombia.',
 'Univ Antioquia, Colombia.']

In [1397]:
fwp.extractOne(aff,raf[extra_right_on].loc[0],scorer=fuzz.ratio)

('Univ Antioquia UdeA, Inst Matemat, Fac Ciencias Exactas & Nat, Medellin, Colombia.',
 70)

In [1398]:
#if raff<cutoff_extra:
Q=Q-0.1
fwp.extractOne(aff,raf[extra_right_on].loc[0],scorer=fuzz.partial_token_set_ratio)

('Univ Antioquia UdeA, Inst Matemat, Fac Ciencias Exactas & Nat, Medellin, Colombia.',
 100)

In [1400]:
#fwp.extractOne(aff,raf[extra_right_on].loc[0],scorer=fuzz.token_sort_ratio)
#fwp.extractOne(aff,raf[extra_right_on].loc[0],scorer=fuzz.token_set_ratio)
#fwp.extractOne(aff,raf[extra_right_on].loc[0],scorer=fuzz.partial_token_sort_ratio)

# ================

Journal:

In [1401]:
#if Q<1:
so=dfnot['SO'].loc[0]

In [1402]:
so

'VARIATIONAL REGULARIZATION OF 3D DATA: EXPERIMENTS WITH MATLAB'

In [1403]:
full_name=raf['UDEA_authors'].loc[0].get('full_name')

In [1404]:
full_name

'MONTEGRANARIO RIASCOS HEBERT'

In [1405]:
if full_name:
    kkk=UDEA[UDEA['UDEA_nombre'].str.contains(full_name)].reset_index(drop=True)

In [1406]:
kkk.SO

0    APPLIED MATHEMATICS AND COMPUTATION
1      SUPERLATTICES AND MICROSTRUCTURES
Name: SO, dtype: object

In [1407]:
fwp.extractOne( so,   kkk.SO, scorer=scorer)

('APPLIED MATHEMATICS AND COMPUTATION', 40, 0)

In [1408]:
#60

# ================

In [1409]:
dfnot=qq#UDEA_NOT.copy()
dfnot=dfnot.reset_index(drop=True)

In [1410]:
l=dfnot['authors_WOS'].loc[0]

In [1426]:
l

[{'WOS_author': 'Montegranario, Hebert',
  'affiliation': ['Univ Antioquia, Math Inst, Medellin, Colombia.'],
  'i': 0}]

In [1427]:
so=dfnot['SO'].loc[0]
so

'VARIATIONAL REGULARIZATION OF 3D DATA: EXPERIMENTS WITH MATLAB'

In [1537]:
#for i in range(20):
#l=dfnot['authors_WOS'].loc[i]
#93,70
from IPython.display import clear_output
def json_fuzzy_merge(l,so,contents,right_target='UDEA_authors',
                       left_on='WOS_author',extra_left_on='affiliation',
                       right_on='WOS_author',extra_right_on='WOS_affiliation',
                       extra_extra_right_on='full_name',
                       cutoff=93,cutoff_extra=70,scorer=fuzz.token_set_ratio,
                       DEBUG=False):
    newl=[]
    newl=[]
    for d in l:
        clear_output(wait=True)
        AUTHOR=False
        AFFILIATION=False
        JOURNAL=True

        dfraf=pd.DataFrame()
        au=d.get(left_on)
        print(au)
        aff=d.get(extra_left_on)[0]
        Q=1
        # Try match author to a good degree
        rau=fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=scorer)
        if DEBUG: print(1,rau)
        if rau[1]>=cutoff:
            AUTHOR=True
            if DEBUG: print(2,AUTHOR)            
        #Try match author with less quality: Q
        else:
            rau=fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),
                       scorer=fuzz.partial_token_sort_ratio)
            if DEBUG: print(2.1,rau)            
            if rau and rau[1]>=cutoff:
                Q=Q-0.1
                AUTHOR=True
        if AUTHOR:
            dfraf=contents[contents[right_on].apply( lambda l: rau[0] in l )
                                ].reset_index(drop=True)
            raf=fwp.extractOne(aff,dfraf[extra_right_on].loc[0],scorer=fuzz.ratio)
            if DEBUG: print(3,rau)
            if raf and raf[1]>=cutoff_extra:
                AFFILIATION=True
            else:
                Q=Q-0.1
                raf=fwp.extractOne(aff,dfraf[extra_right_on].loc[0],
                                   scorer=fuzz.partial_token_set_ratio)
                if raf and raf[1]>=cutoff_extra:
                    AFFILIATION=True

        if DEBUG: print(4,AFFILIATION,Q)                
        if Q<1:
            cutoff_so=50
            if Q<0.9:
                cutoff_so=60
            if not dfraf.empty:
                full_name=dfraf[right_target].loc[0].get(
                        extra_extra_right_on)
                if full_name:
                    kkk=UDEA[UDEA['UDEA_nombre'].str.contains(full_name)
                                ].reset_index(drop=True)
                    rso=fwp.extractOne( so,   kkk.SO, scorer=scorer)
                    if not rso:
                        JOURNAL=False
                    elif rso[1]<cutoff_so:
                        JOURNAL=False
            else:
                JOURNAL=False
        if DEBUG: print(5,JOURNAL)                
        if AUTHOR and AFFILIATION and JOURNAL:
            newl=newl+[  dfraf.loc[0,right_target]  ]
    if newl:
        return newl
    else:
        return None

In [1538]:
dfraf=json_fuzzy_merge(l,so,contents,DEBUG=True)

Montegranario, Hebert
1 ('Montegranario, H.', 93)
2 True
3 ('Montegranario, H.', 93)
4 True 1
5 True


In [ ]:
kk=UDEA_NOT['authors_WOS'].combine('SO',func=lambda l,so: json_fuzzy_merge(l,so,contents) 
                                   if type(l)==list else None
                                  )

Lemos, Juan D.


In [1342]:
#:
RESET=False

print(tmprau)
if RESET:
    Q=Q-0.1
    rau=tmprau

('Sanchez, H.', 89)


In [1343]:
#fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=fuzz.token_sort_ratio)
#fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=fuzz.ratio)#
#fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=fuzz.partial_token_set_ratio)

In [1353]:
full_name

'SANCHEZ LONDOÑO HECTOR DARIO'

In [1354]:
%loa

In [1054]:
#60